In [5]:
#pip install torch==1.10.0+cu113 torchvision==0.11.1+cu113 torchaudio===0.10.0+cu113 -f https://download.pytorch.org/whl/cu113/torch_stable.html

Looking in links: https://download.pytorch.org/whl/cu113/torch_stable.htmlNote: you may need to restart the kernel to use updated packages.

  Attempting uninstall: torch
    Found existing installation: torch 1.4.0+cu92
    Uninstalling torch-1.4.0+cu92:
      Successfully uninstalled torch-1.4.0+cu92
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.11.1
    Uninstalling torchvision-0.11.1:
      Successfully uninstalled torchvision-0.11.1
  Attempting uninstall: torchaudio
    Found existing installation: torchaudio 0.10.0
    Uninstalling torchaudio-0.10.0:
      Successfully uninstalled torchaudio-0.10.0


Test Pytorch

In [1]:
import torch
x = torch.rand(5, 3)
print(x)

tensor([[0.9752, 0.2226, 0.2277],
        [0.2493, 0.1693, 0.7396],
        [0.7140, 0.3910, 0.2681],
        [0.8534, 0.7624, 0.1596],
        [0.5297, 0.4789, 0.5907]])


## gen_trainval_list.py

This portion of the code seems to be only writing the image name and its corresponding mat files into a txt file.

In [1]:
import os
import glob
import random

In [2]:
root = 'maize_counting_dataset'
image_folder = 'images'
label_folder = 'labels'
train = 'train'
val = 'val'

In [3]:
train_path = os.path.join(root, train)
with open('train.txt', 'w') as f:
    for image_path in glob.glob(os.path.join(train_path, image_folder, '*.JPG')):
        im_path = image_path.replace(root, '')
        gt_path = im_path.replace(image_folder, label_folder).replace('.JPG', '.xml')
        f.write(im_path+'\t'+gt_path+'\n')

In [4]:
val_path = os.path.join(root, val)
with open('val.txt', 'w') as f:
    for image_path in glob.glob(os.path.join(val_path, image_folder, '*.JPG')):
        im_path = image_path.replace(root, '')
        gt_path = im_path.replace(image_folder, label_folder).replace('.JPG', '.xml')
        f.write(im_path+'\t'+gt_path+'\n')

## hldataset.py

In [5]:
import os
import json
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import pandas as pd 
import random
import numpy as np
from PIL import Image
import cv2
import h5py
import scipy.io as sio
from scipy.ndimage.filters import gaussian_filter
from skimage import util
from skimage.measure import label
from skimage.measure import regionprops
import xml.etree.ElementTree as ET

import torch
from torch.utils.data import Dataset
from torchvision import transforms
import torch.nn.functional as F

In [6]:
def read_image(x):
    img_arr = np.array(Image.open(x))
    if len(img_arr.shape) == 2:  # grayscale
        img_arr = np.tile(img_arr, [3, 1, 1]).transpose(1, 2, 0)
    return img_arr

In [7]:
class RandomCrop(object):
    def __init__(self, output_size):
        assert isinstance(output_size, (int, tuple))
        self.output_size = output_size

    def __call__(self, sample):

        image, target, gtcount = sample['image'], sample['target'], sample['gtcount']
        h, w = image.shape[:2]

        if isinstance(self.output_size, tuple):
            new_h = min(self.output_size[0], h)
            new_w = min(self.output_size[1], w)
            assert (new_h, new_w) == self.output_size
        else:
            crop_size = min(self.output_size, h, w)
            assert crop_size == self.output_size
            new_h = new_w = crop_size
        if gtcount > 0:
            mask = target > 0
            ch, cw = int(np.ceil(new_h / 2)), int(np.ceil(new_w / 2))
            mask_center = np.zeros((h, w), dtype=np.uint8)
            mask_center[ch:h-ch+1, cw:w-cw+1] = 1
            mask = (mask & mask_center)
            idh, idw = np.where(mask == 1)
            if len(idh) != 0:
                ids = random.choice(range(len(idh)))
                hc, wc = idh[ids], idw[ids]
                top, left = hc-ch, wc-cw
            else:
                top = np.random.randint(0, h-new_h+1)
                left = np.random.randint(0, w-new_w+1)
        else:
            top = np.random.randint(0, h-new_h+1)
            left = np.random.randint(0, w-new_w+1)

        image = image[top:top+new_h, left:left+new_w, :]
        target = target[top:top+new_h, left:left+new_w]

        return {'image': image, 'target': target, 'gtcount': gtcount}

In [8]:
class RandomFlip(object):
    def __init__(self):
        pass

    def __call__(self, sample):
        image, target, gtcount = sample['image'], sample['target'], sample['gtcount']
        do_mirror = np.random.randint(2)
        if do_mirror:
            image = cv2.flip(image, 1)
            target = cv2.flip(target, 1)
        return {'image': image, 'target': target, 'gtcount': gtcount}

In [9]:
class Normalize(object):

    def __init__(self, scale, mean, std):
        self.scale = scale
        self.mean = mean
        self.std = std

    def __call__(self, sample):
        image, target, gtcount = sample['image'], sample['target'], sample['gtcount']
        image, target = image.astype('float32'), target.astype('float32')

        # pixel normalization
        image = (self.scale * image - self.mean) / self.std

        image, target = image.astype('float32'), target.astype('float32')

        return {'image': image, 'target': target, 'gtcount': gtcount}

In [10]:
class ZeroPadding(object):
    def __init__(self, psize=32):
        self.psize = psize

    def __call__(self, sample):
        psize =  self.psize

        image, target, gtcount = sample['image'], sample['target'], sample['gtcount']
        h,w = image.size()[-2:]
        ph,pw = (psize-h%psize),(psize-w%psize)
        # print(ph,pw)

        (pl, pr) = (pw//2, pw-pw//2) if pw != psize else (0, 0)
        (pt, pb) = (ph//2, ph-ph//2) if ph != psize else (0, 0)
        if (ph!=psize) or (pw!=psize):
            tmp_pad = [pl, pr, pt, pb]
            # print(tmp_pad)
            image = F.pad(image,tmp_pad)
            target = F.pad(target,tmp_pad)

        return {'image': image, 'target': target, 'gtcount': gtcount}

In [11]:
class ToTensor(object):
    """Convert ndarrays in sample to Tensors."""

    def __init__(self):
        pass

    def __call__(self, sample):
        # swap color axis
        # numpy image: H x W x C
        # torch image: C X H X W
        image, target, gtcount = sample['image'], sample['target'], sample['gtcount']
        image = image.transpose((2, 0, 1))
        target = np.expand_dims(target, axis=2)
        target = target.transpose((2, 0, 1))
        image, target = torch.from_numpy(image), torch.from_numpy(target)
        return {'image': image, 'target': target, 'gtcount': gtcount}

In [12]:
class MaizeTasselDataset(Dataset):
    def __init__(self, data_dir, data_list, ratio, train=True, transform=None):
        self.data_dir = data_dir
        self.data_list = [name.split('\t') for name in open(data_list).read().splitlines()]
        self.ratio = ratio
        self.train = train
        self.transform = transform
        self.image_list = []
        
        # store images and generate ground truths
        self.images = {}
        self.targets = {}
        self.gtcounts = {}
        self.dotimages = {}

    def bbs2points(self, bbs):    
        points = []
        for bb in bbs:
            x1, y1, w, h = [float(b) for b in bb]
            x2, y2 = x1+w-1, y1+h-1
            x, y = np.round((x1+x2)/2).astype(np.int32), np.round((y1+y2)/2).astype(np.int32)
            points.append([x, y])
        return points

    def __len__(self):
        return len(self.data_list)

    def __getitem__(self, idx):
        file_name = self.data_list[idx]
        self.image_list.append(file_name[0])
        if file_name[0] not in self.images:
            image = read_image(self.data_dir+file_name[0])
            annotation = sio.loadmat(self.data_dir+file_name[1])
            h, w = image.shape[:2]
            nh = int(np.ceil(h * self.ratio))
            nw = int(np.ceil(w * self.ratio))
            image = cv2.resize(image, (nw, nh), interpolation = cv2.INTER_CUBIC)
            target = np.zeros((nh, nw), dtype=np.float32)
            dotimage = image.copy()
            if annotation['annotation'][0][0][1] is not None:
                bbs = annotation['annotation'][0][0][1]
                gtcount = bbs.shape[0]
                pts = self.bbs2points(bbs)
                for pt in pts:
                    pt[0], pt[1] = int(pt[0] * self.ratio), int(pt[1] * self.ratio)
                    target[pt[1], pt[0]] = 1
                    cv2.circle(dotimage, (pt[0], pt[1]), int(24 * self.ratio) , (255, 0, 0), -1)
            else:
                gtcount = 0
            target = gaussian_filter(target, 80 * self.ratio)

            # plt.imshow(target, cmap=cm.jet)
            # plt.show()
            # print(target.sum())

            self.images.update({file_name[0]:image})
            self.targets.update({file_name[0]:target})
            self.gtcounts.update({file_name[0]:gtcount})
            self.dotimages.update({file_name[0]:dotimage})

        
        sample = {
            'image': self.images[file_name[0]], 
            'target': self.targets[file_name[0]], 
            'gtcount': self.gtcounts[file_name[0]]
        }

        if self.transform:
            sample = self.transform(sample)

        return sample

In [13]:
dataset = MaizeTasselDataset(
        data_dir='maize_counting_dataset', 
        data_list='maize_counting_dataset/train.txt',
        ratio=0.167, 
        train=True, 
        transform=transforms.Compose([
            ToTensor()]
        )
    )

In [14]:
dataloader = torch.utils.data.DataLoader(
        dataset,
        batch_size=1,
        shuffle=False,
        num_workers=0
    )

In [15]:
print(len(dataloader))
mean = 0.
std = 0.
for i, data in enumerate(dataloader, 0):
        images, targets = data['image'], data['target']
        bs = images.size(0)
        images = images.view(bs, images.size(1), -1).float()
        mean += images.mean(2).sum(0)
        std += images.std(2).sum(0)
        print(images.size())
        print(i) 
mean /= len(dataloader)
std /= len(dataloader)
print(mean/255.)
print(std/255.)

186
torch.Size([1, 3, 278770])
0
torch.Size([1, 3, 278770])
1
torch.Size([1, 3, 278770])
2
torch.Size([1, 3, 278770])
3
torch.Size([1, 3, 278770])
4
torch.Size([1, 3, 222530])
5
torch.Size([1, 3, 278770])
6
torch.Size([1, 3, 222530])
7
torch.Size([1, 3, 278770])
8
torch.Size([1, 3, 278770])
9
torch.Size([1, 3, 278770])
10
torch.Size([1, 3, 278770])
11
torch.Size([1, 3, 278770])
12
torch.Size([1, 3, 278770])
13
torch.Size([1, 3, 278770])
14
torch.Size([1, 3, 278770])
15
torch.Size([1, 3, 278770])
16
torch.Size([1, 3, 278770])
17
torch.Size([1, 3, 278770])
18
torch.Size([1, 3, 278770])
19
torch.Size([1, 3, 278770])
20
torch.Size([1, 3, 278770])
21
torch.Size([1, 3, 278770])
22
torch.Size([1, 3, 278770])
23
torch.Size([1, 3, 278770])
24
torch.Size([1, 3, 278770])
25
torch.Size([1, 3, 278770])
26
torch.Size([1, 3, 278770])
27
torch.Size([1, 3, 278770])
28
torch.Size([1, 3, 278770])
29
torch.Size([1, 3, 278770])
30
torch.Size([1, 3, 278770])
31
torch.Size([1, 3, 278770])
32
torch.Size([1, 3

## utils.py

In [16]:
import scipy
import numpy as np
import math
import cv2 as cv
from scipy.ndimage import gaussian_filter, morphology
from skimage.measure import label, regionprops
from sklearn import linear_model
import matplotlib.pyplot as plt

In [17]:
def compute_mae(pd, gt):
    pd, gt = np.array(pd), np.array(gt)
    diff = pd - gt
    mae = np.mean(np.abs(diff))
    return mae


def compute_mse(pd, gt):
    pd, gt = np.array(pd), np.array(gt)
    diff = pd - gt
    mse = np.sqrt(np.mean((diff ** 2)))
    return mse

def compute_relerr(pd, gt):
    pd, gt = np.array(pd), np.array(gt)
    diff = pd - gt
    diff = diff[gt > 0]
    gt = gt[gt > 0]
    if (diff is not None) and (gt is not None):
        rmae = np.mean(np.abs(diff) / gt) * 100
        rmse = np.sqrt(np.mean(diff**2 / gt**2)) * 100
    else:
        rmae = 0
        rmse = 0
    return rmae, rmse


def rsquared(pd, gt):
    """ Return R^2 where x and y are array-like."""
    pd, gt = np.array(pd), np.array(gt)
    slope, intercept, r_value, p_value, std_err = scipy.stats.linregress(pd, gt)
    return r_value**2


def dense_sample2d(x, sx, stride):
    (h,w) = x.shape[:2]
    #idx_img = np.array([i for i in range(h*w)]).reshape(h,w)
    idx_img = np.zeros((h,w),dtype=float)
    
    th = [i for i in range(0, h-sx+1, stride)]
    tw = [j for j in range(0, w-sx+1, stride)]
    norm_vec = np.zeros(len(th)*len(tw))

    for i in th:
        for j in tw:
            idx_img[i:i+sx,j:j+sx] = idx_img[i:i+sx,j:j+sx]+1

    # plot redundancy map
    import os
    import matplotlib.pyplot as plt
    cmap = plt.cm.get_cmap('hot')
    idx_img = idx_img / (idx_img.max())
    idx_img = cmap(idx_img) * 255.
    plt.figure()
    plt.imshow(idx_img.astype(np.uint8))
    plt.axis('off')
    plt.savefig(os.path.join('redundancy_map.pdf'), bbox_inches='tight', dpi = 300)
    plt.close()
   
    idx_img = 1/idx_img
    idx_img = idx_img/sx/sx
    #line order
    idx = 0
    for i in th:
        for j in tw:
            norm_vec[idx] =idx_img[i:i+sx,j:j+sx].sum()
            idx+=1
    
    return norm_vec


def recover_countmap(pred, image, patch_sz, stride):
    pred = pred.reshape(-1)
    imH, imW = image.shape[2:4]
    cntMap = np.zeros((imH, imW), dtype=float)
    norMap = np.zeros((imH, imW), dtype=float)
    
    H = np.arange(0, imH - patch_sz + 1, stride)
    W = np.arange(0, imW - patch_sz + 1, stride)
    cnt = 0
    for h in H:
        for w in W:
            pixel_cnt = pred[cnt] / patch_sz / patch_sz
            cntMap[h:h+patch_sz, w:w+patch_sz] += pixel_cnt
            norMap[h:h+patch_sz, w:w+patch_sz] += np.ones((patch_sz,patch_sz))
            cnt += 1
    return cntMap / (norMap + 1e-12)

## hlnet.py

In [18]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [19]:
class Encoder(nn.Module):
    def __init__(self, arc='tasselnetv2'):
        super(Encoder, self).__init__()
        if arc == 'tasselnetv2':
            self.encoder = nn.Sequential(
                nn.Conv2d(3, 16, 3, padding=1, bias=False),
                nn.BatchNorm2d(16),
                nn.ReLU(inplace=True),
                nn.MaxPool2d((2, 2), stride=2),
                nn.Conv2d(16, 32, 3, padding=1, bias=False),
                nn.BatchNorm2d(32),
                nn.ReLU(inplace=True),
                nn.MaxPool2d((2, 2), stride=2),
                nn.Conv2d(32, 64, 3, padding=1, bias=False),
                nn.BatchNorm2d(64),
                nn.ReLU(inplace=True),
                nn.MaxPool2d((2, 2), stride=2),
                nn.Conv2d(64, 128, 3, padding=1, bias=False),
                nn.BatchNorm2d(128),
                nn.ReLU(inplace=True),
                nn.Conv2d(128, 128, 3, padding=1, bias=False),
                nn.BatchNorm2d(128),
                nn.ReLU(inplace=True),
            )
        elif arc == 'tasselnetv2plus':
            self.encoder = nn.Sequential(
                nn.Conv2d(3, 16, 3, padding=1, bias=False),
                nn.BatchNorm2d(16),
                nn.ReLU(inplace=True),
                nn.MaxPool2d((2, 2), stride=2),
                nn.Conv2d(16, 32, 3, padding=1, bias=False),
                nn.BatchNorm2d(32),
                nn.ReLU(inplace=True),
                nn.MaxPool2d((2, 2), stride=2),
                nn.Conv2d(32, 64, 3, padding=1, bias=False),
                nn.BatchNorm2d(64),
                nn.ReLU(inplace=True),
                nn.MaxPool2d((2, 2), stride=2),
                nn.Conv2d(64, 128, 3, padding=1, bias=False),
                nn.BatchNorm2d(128),
                nn.ReLU(inplace=True),
                nn.Conv2d(128, 128, 3, padding=1, bias=False),
                nn.BatchNorm2d(128),
                nn.ReLU(inplace=True)
            )
        else:
            raise NotImplementedError

    def forward(self, x):
        x = self.encoder(x)
        return x

In [20]:
class Counter(nn.Module):
    def __init__(self, arc='tasselnetv2', input_size=64, output_stride=8):
        super(Counter, self).__init__()
        k = int(input_size / 8)
        avg_pool_stride = int(output_stride / 8)

        if arc == 'tasselnetv2':
            self.counter = nn.Sequential(
                nn.Conv2d(128, 128, (k, k), bias=False),
                nn.BatchNorm2d(128),
                nn.ReLU(inplace=True),
                nn.Conv2d(128, 128, 1, bias=False),
                nn.BatchNorm2d(128),
                nn.ReLU(inplace=True),
                nn.Conv2d(128, 1, 1)
            )
        elif arc == 'tasselnetv2plus':
            self.counter = nn.Sequential(
                nn.AvgPool2d((k, k), stride=avg_pool_stride),
                nn.Conv2d(128, 128, 1, bias=False),
                nn.BatchNorm2d(128),
                nn.ReLU(inplace=True),
                nn.Conv2d(128, 1, 1)
            )
        else:
            raise NotImplementedError

    def forward(self, x):
        x = self.counter(x)
        return x

In [21]:
class Normalizer:
    @staticmethod
    def cpu_normalizer(x, imh, imw, insz, os):
        # CPU normalization
        bs = x.size()[0]
        normx = np.zeros((imh, imw))
        norm_vec = dense_sample2d(normx, insz, os).astype(np.float32)
        x = x.cpu().detach().numpy().reshape(bs, -1) * norm_vec
        return x
    
    @staticmethod
    def gpu_normalizer(x, imh, imw, insz, os):
        _, _, h, w = x.size()            
        accm = torch.cuda.FloatTensor(1, insz*insz, h*w).fill_(1)           
        accm = F.fold(accm, (imh, imw), kernel_size=insz, stride=os)
        accm = 1 / accm
        accm /= insz**2
        accm = F.unfold(accm, kernel_size=insz, stride=os).sum(1).view(1, 1, h, w)
        x *= accm
        return x.squeeze().cpu().detach().numpy()

In [22]:
class CountingModels(nn.Module):
    def __init__(self, arc='tasselnetv2', input_size=64, output_stride=8):
        super(CountingModels, self).__init__()
        self.input_size = input_size
        self.output_stride = output_stride

        self.encoder = Encoder(arc)
        self.counter = Counter(arc, input_size, output_stride)
        if arc == 'tasselnetv2':
            self.normalizer = Normalizer.cpu_normalizer
        elif arc == 'tasselnetv2plus':
            self.normalizer = Normalizer.gpu_normalizer
        
        self.weight_init()

    def forward(self, x, is_normalize=True):
        imh, imw = x.size()[2:]
        x = self.encoder(x)
        x = self.counter(x)
        if is_normalize:
            x = self.normalizer(x, imh, imw, self.input_size, self.output_stride)
        return x

    def weight_init(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.normal_(m.weight, std=0.01)
                # nn.init.kaiming_uniform_(
                #         m.weight, 
                #         mode='fan_in', 
                #         nonlinearity='relu'
                #         )
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

In [23]:
from time import time

insz, os = 64, 8
imH, imW = 1080, 1920
#net = CountingModels(arc='tasselnetv2', input_size=insz, output_stride=os).cuda()
net = CountingModels(arc='tasselnetv2', input_size=insz, output_stride=os)
with torch.no_grad():
    net.eval()
    #x = torch.randn(1, 3, imH, imW).cuda()
    x = torch.randn(1, 3, imH, imW)
    y = net(x)
    print(y.shape)    

with torch.no_grad():
    frame_rate = np.zeros((100, 1))
    for i in range(100):
        #x = torch.randn(1, 3, imH, imW).cuda()
        x = torch.randn(1, 3, imH, imW)
        #torch.cuda.synchronize()
        start = time()

        y = net(x)

        #torch.cuda.synchronize()
        end = time()

        running_frame_rate = 1 * float(1 / (end - start))
        frame_rate[i] = running_frame_rate
    print(np.mean(frame_rate))

<ipython-input-17-9a25d3e4bf17>:60: RuntimeWarning: divide by zero encountered in true_divide
  idx_img = 1/idx_img


(1, 29824)
0.4462365785926013


## hltrainval.py

In [24]:
import os
import argparse
from time import time

import cv2 as cv
import numpy as np
from PIL import Image
from matplotlib import pyplot as plt

plt.switch_backend('agg')
# from skimage.measure import compare_psnr updated to peak_signal_noise_ratio
# from skimage.measure import compare_ssim updated to structural_similarity
from skimage.metrics import peak_signal_noise_ratio
from skimage.metrics import structural_similarity
import torch.backends.cudnn as cudnn

import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import StepLR, MultiStepLR

In [25]:
# prevent dataloader deadlock, uncomment if deadlock occurs
# cv.setNumThreads(0)
#cudnn.enabled = True

# constant
IMG_SCALE = 1. / 255
IMG_MEAN = [.3405, .4747, .2418]
IMG_STD = [1, 1, 1]
SCALES = [0.7, 1, 1.3]
SHORTER_SIDE = 224

# system-related parameters
DATA_DIR = 'maize_counting_dataset'
DATASET = 'mtc'
EXP = 'tasselnetv2_rf110_i64o8_r0125_crop256_lr-2_bs9_epoch500'
DATA_LIST = 'maize_counting_dataset/train.txt'
DATA_VAL_LIST = 'maize_counting_dataset/test.txt'

RESTORE_FROM = 'model_best.pth.tar'
SNAPSHOT_DIR = './snapshots'
RESULT_DIR = './results'

# model-related parameters
INPUT_SIZE = 64
OUTPUT_STRIDE = 8
MODEL = 'tasselnetv2'
RESIZE_RATIO = 0.125

# training-related parameters
OPTIMIZER = 'sgd'  # choice in ['sgd', 'adam']
BATCH_SIZE = 9
CROP_SIZE = (256, 256)
LEARNING_RATE = 1e-2
MILESTONES = [200, 400]
MOMENTUM = 0.95
MULT = 1
NUM_EPOCHS = 500
NUM_CPU_WORKERS = 0
PRINT_EVERY = 1
RANDOM_SEED = 6
WEIGHT_DECAY = 5e-4
VAL_EVERY = 1

device = torch.device("cpu")
#device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# add a new entry here if creating a new data loader
dataset_list = {
    'mtc': MaizeTasselDataset
}

In [ ]:
def get_arguments():
    """Parse all the arguments provided from the CLI.

    Returns:
      A list of parsed arguments.
    """
    parser = argparse.ArgumentParser(description="Object Counting Framework")
    # constant
    parser.add_argument("--image-scale", type=float, default=IMG_SCALE, help="Scale factor used in normalization.")
    parser.add_argument("--image-mean", nargs='+', type=float, default=IMG_MEAN, help="Mean used in normalization.")
    parser.add_argument("--image-std", nargs='+', type=float, default=IMG_STD, help="Std used in normalization.")
    parser.add_argument("--scales", type=int, default=SCALES, help="Scales of crop.")
    parser.add_argument("--shorter-side", type=int, default=SHORTER_SIDE, help="Shorter side of the image.")
    # system-related parameters
    parser.add_argument("--data-dir", type=str, default=DATA_DIR, help="Path to the directory containing the dataset.")
    parser.add_argument("--dataset", type=str, default=DATASET, help="Dataset type.")
    parser.add_argument("--exp", type=str, default=EXP, help="Experiment path.")
    parser.add_argument("--data-list", type=str, default=DATA_LIST,
                        help="Path to the file listing the images in the dataset.")
    parser.add_argument("--data-val-list", type=str, default=DATA_VAL_LIST,
                        help="Path to the file listing the images in the val dataset.")
    parser.add_argument("--restore-from", type=str, default=RESTORE_FROM, help="Name of restored model.")
    parser.add_argument("--snapshot-dir", type=str, default=SNAPSHOT_DIR, help="Where to save snapshots of the model.")
    parser.add_argument("--result-dir", type=str, default=RESULT_DIR, help="Where to save inferred results.")
    parser.add_argument("--save-output", action="store_true", help="Whether to save the output.")
    # model-related parameters
    parser.add_argument("--input-size", type=int, default=INPUT_SIZE, help="the minimum input size of the model.")
    parser.add_argument("--output-stride", type=int, default=OUTPUT_STRIDE, help="Output stride of the model.")
    parser.add_argument("--resize-ratio", type=float, default=RESIZE_RATIO, help="Resizing ratio.")
    parser.add_argument("--model", type=str, default=MODEL, help="model to be chosen.")
    parser.add_argument("--use-pretrained", action="store_true", help="Whether to use pretrained model.")
    parser.add_argument("--freeze-bn", action="store_true", help="Whether to freeze encoder bnorm layers.")
    parser.add_argument("--sync-bn", action="store_true", help="Whether to apply synchronized batch normalization.")
    # training-related parameters
    parser.add_argument("--optimizer", type=str, default=OPTIMIZER, choices=['sgd', 'adam'], help="Choose optimizer.")
    parser.add_argument("--batch-size", type=int, default=BATCH_SIZE,
                        help="Number of images sent to the network in one step.")
    parser.add_argument("--milestones", nargs='+', type=int, default=MILESTONES, help="Multistep policy.")
    parser.add_argument("--crop-size", nargs='+', type=int, default=CROP_SIZE, help="Size of crop.")
    parser.add_argument("--evaluate-only", action="store_true", help="Whether to perform evaluation.")
    parser.add_argument("--learning-rate", type=float, default=LEARNING_RATE, help="Base learning rate for training.")
    parser.add_argument("--momentum", type=float, default=MOMENTUM, help="Momentum component of the optimizer.")
    parser.add_argument("--weight-decay", type=float, default=WEIGHT_DECAY,
                        help="Regularisation parameter for L2-loss.")
    parser.add_argument("--mult", type=float, default=MULT, help="LR multiplier for pretrained layers.")
    parser.add_argument("--num-epochs", type=int, default=NUM_EPOCHS, help="Number of training epochs.")
    parser.add_argument("--num-workers", type=int, default=NUM_CPU_WORKERS, help="Number of CPU cores used.")
    parser.add_argument("--print-every", type=int, default=PRINT_EVERY, help="Print information every often.")
    parser.add_argument("--random-seed", type=int, default=RANDOM_SEED,
                        help="Random seed to have reproducible results.")
    parser.add_argument("--val-every", type=int, default=VAL_EVERY, help="How often performing validation.")
    return parser.parse_args()



In [26]:
def save_checkpoint(state, snapshot_dir, filename='model_ckpt.pth.tar'):
    torch.save(state, '{}/{}'.format(snapshot_dir, filename))


def plot_learning_curves(net, dir_to_save):
    # plot learning curves
    fig = plt.figure(figsize=(16, 9))
    ax1 = fig.add_subplot(1, 2, 1)
    ax1.plot(net.train_loss['epoch_loss'], label='train loss', color='tab:blue')
    ax1.legend(loc='upper right')
    ax2 = fig.add_subplot(1, 2, 2)
    ax2.plot(net.val_loss['epoch_loss'], label='val mae', color='tab:orange')
    ax2.legend(loc='upper right')
    # ax2.set_ylim((0,50))
    fig.savefig(os.path.join(dir_to_save, 'learning_curves.png'), bbox_inches='tight', dpi=300)
    plt.close()

#### AFTER EDIT

In [44]:
def train(net, train_loader, criterion, optimizer, epoch, args):
    # switch to 'train' mode
    net.train()

    # uncomment the following line if the training images don't have the same size
    cudnn.benchmark = True

    if BATCH_SIZE == 1:
        for m in net.modules():
            if isinstance(m, nn.BatchNorm2d):
                m.eval()

    running_loss = 0.0
    avg_frame_rate = 0.0
    in_sz = INPUT_SIZE
    os = OUTPUT_STRIDE
    #target_filter = torch.cuda.FloatTensor(1, 1, in_sz, in_sz).fill_(1)
    target_filter = torch.FloatTensor(1, 1, in_sz, in_sz).fill_(1)
    for i, sample in enumerate(train_loader):
        #torch.cuda.synchronize()
        start = time()

        inputs, targets = sample['image'], sample['target']
        #inputs, targets = inputs.cuda(), targets.cuda()

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward
        outputs = net(inputs, is_normalize=False)
        # generate targets
        targets = F.conv2d(targets, target_filter, stride=os)
        # compute loss
        loss = criterion(outputs, targets)

        # backward + optimize
        loss.backward()
        optimizer.step()
        # collect and print statistics
        running_loss += loss.item()

        #torch.cuda.synchronize()
        end = time()

        running_frame_rate = BATCH_SIZE * float(1 / (end - start))
        avg_frame_rate = (avg_frame_rate * i + running_frame_rate) / (i + 1)
        if i % PRINT_EVERY == PRINT_EVERY - 1:
            print('epoch: %d, train: %d/%d, '
                  'loss: %.5f, frame: %.2fHz/%.2fHz' % (
                      epoch,
                      i + 1,
                      len(train_loader),
                      running_loss / (i + 1),
                      running_frame_rate,
                      avg_frame_rate
                  ))
    net.train_loss['epoch_loss'].append(running_loss / (i + 1))

In [40]:
def validate(net, valset, val_loader, criterion, epoch, args):
    # switch to 'eval' mode
    net.eval()
    cudnn.benchmark = False

    image_list = valset.image_list

    #if args.save_output:
     #   epoch_result_dir = os.path.join(args.result_dir, str(epoch))
      #  if not os.path.exists(epoch_result_dir):
       #     os.makedirs(epoch_result_dir)
        #cmap = plt.cm.get_cmap('jet')
    epoch_result_dir = os.path.join(RESULT_DIR, str(epoch))
    if not os.path.exists(epoch_result_dir):
        os.makedirs(epoch_result_dir)
    cmap = plt.cm.get_cmap('jet')
    
    pd_counts = []
    gt_counts = []
    with torch.no_grad():
        avg_frame_rate = 0.0
        for i, sample in enumerate(val_loader):
            #torch.cuda.synchronize()
            start = time()

            image, gtcount = sample['image'], sample['gtcount']
            # inference
            #output = net(image.cuda(), is_normalize=not args.save_output)
            output = net(image, is_normalize = False)
            #if args.save_output:
            #    output_save = output
                # normalization
            #    output = Normalizer.gpu_normalizer(output, image.size()[2], image.size()[3], args.input_size,
            #                                       args.output_stride)
            output_save = output
            output = Normalizer.cpu_normalizer(output, image.size()[2], image.size()[3], INPUT_SIZE,
                                                   OUTPUT_STRIDE)
            
            # postprocessing
            output = np.clip(output, 0, None)

            pdcount = output.sum()
            gtcount = float(gtcount.numpy())

            #if args.save_output:
            x = True
            if x:
                _, image_name = os.path.split(image_list[i])
                output_save = np.clip(output_save.squeeze().cpu().numpy(), 0, None)
                output_save = recover_countmap(output_save, image, INPUT_SIZE, OUTPUT_STRIDE)
                output_save = output_save / (output_save.max() + 1e-12)
                output_save = cmap(output_save) * 255.
                # image composition
                image = valset.images[image_list[i]]
                nh, nw = output_save.shape[:2]
                image = cv2.resize(image, (nw, nh), interpolation=cv2.INTER_CUBIC)
                output_save = 0.5 * image + 0.5 * output_save[:, :, 0:3]

                dotimage = valset.dotimages[image_list[i]]

                fig = plt.figure()
                ax1 = fig.add_subplot(1, 2, 1)
                ax1.imshow(dotimage.astype(np.uint8))
                ax1.get_xaxis().set_visible(False)
                ax1.get_yaxis().set_visible(False)
                ax2 = fig.add_subplot(1, 2, 2)
                ax2.imshow(output_save.astype(np.uint8))
                ax2.get_xaxis().set_visible(False)
                ax2.get_yaxis().set_visible(False)
                fig.suptitle('manual count=%4.2f, inferred count=%4.2f' % (gtcount, pdcount), fontsize=10)
                if DATASET == 'mtc':
                    plt.tight_layout(rect=[0, 0, 1, 1.4])  # maize tassels counting
                elif DATASET == 'wec':
                    plt.tight_layout(rect=[0, 0, 1, 1.45])  # wheat ears counting
                elif DATASET == 'shc':
                    plt.tight_layout(rect=[0, 0, 0.95, 1])  # sorghum heads counting -- dataset1
                    # plt.tight_layout(rect=[0, 0, 1.2, 1]) # sorghum heads counting -- dataset2
                plt.savefig(os.path.join(epoch_result_dir, image_name.replace('.jpg', '.png')), bbox_inches='tight',
                            dpi=300)
                plt.close()

            # compute mae and mse
            pd_counts.append(pdcount)
            gt_counts.append(gtcount)
            mae = compute_mae(pd_counts, gt_counts)
            mse = compute_mse(pd_counts, gt_counts)
            rmae, rmse = compute_relerr(pd_counts, gt_counts)

            #torch.cuda.synchronize()
            end = time()

            running_frame_rate = 1 * float(1 / (end - start))
            avg_frame_rate = (avg_frame_rate * i + running_frame_rate) / (i + 1)
            if i % PRINT_EVERY == PRINT_EVERY - 1:
                print(
                    'epoch: {0}, test: {1}/{2}, pre: {3:.2f}, gt:{4:.2f}, me:{5:.2f}, mae: {6:.2f}, mse: {7:.2f}, rmae: {8:.2f}%, rmse: {9:.2f}%, frame: {10:.2f}Hz/{11:.2f}Hz'
                        .format(epoch, i + 1, len(val_loader), pdcount, gtcount, pdcount - gtcount, mae, mse, rmae,
                                rmse, running_frame_rate, avg_frame_rate)
                )
            start = time()
    r2 = rsquared(pd_counts, gt_counts)
    np.save(SNAPSHOT_DIR + '/pd.npy', pd_counts)
    np.save(SNAPSHOT_DIR + '/gt.npy', gt_counts)
    print('epoch: {0}, mae: {1:.2f}, mse: {2:.2f}, rmae: {3:.2f}%, rmse: {4:.2f}%, r2: {5:.4f}'.format(epoch, mae, mse,
                                                                                                       rmae, rmse, r2))
    # write to files        
    with open(os.path.join(SNAPSHOT_DIR, EXP + '.txt'), 'a') as f:
        print(
            'epoch: {0}, mae: {1:.2f}, mse: {2:.2f}, rmae: {3:.2f}%, rmse: {4:.2f}%, r2: {5:.4f}'.format(epoch, mae,
                                                                                                         mse, rmae,
                                                                                                         rmse, r2),
            file=f
        )
    with open(os.path.join(SNAPSHOT_DIR, 'counts.txt'), 'a') as f:
        for pd, gt in zip(pd_counts, gt_counts):
            print(
                '{0} {1}'.format(pd, gt),
                file=f
            )
    # save stats
    net.val_loss['epoch_loss'].append(mae)
    net.measure['mae'].append(mae)
    net.measure['mse'].append(mse)
    net.measure['rmae'].append(rmae)
    net.measure['rmse'].append(rmse)
    net.measure['r2'].append(r2)


In [59]:
def train(net, train_loader, criterion, optimizer, epoch, args):
    # switch to 'train' mode
    net.train()

    # uncomment the following line if the training images don't have the same size
    cudnn.benchmark = True

    if args.batch_size == 1:
        for m in net.modules():
            if isinstance(m, nn.BatchNorm2d):
                m.eval()

    running_loss = 0.0
    avg_frame_rate = 0.0
    in_sz = args.input_size
    os = args.output_stride
    target_filter = torch.cuda.FloatTensor(1, 1, in_sz, in_sz).fill_(1)
    for i, sample in enumerate(train_loader):
        torch.cuda.synchronize()
        start = time()

        inputs, targets = sample['image'], sample['target']
        inputs, targets = inputs.cuda(), targets.cuda()

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward
        outputs = net(inputs, is_normalize=False)
        # generate targets
        targets = F.conv2d(targets, target_filter, stride=os)
        # compute loss
        loss = criterion(outputs, targets)

        # backward + optimize
        loss.backward()
        optimizer.step()
        # collect and print statistics
        running_loss += loss.item()

        torch.cuda.synchronize()
        end = time()

        running_frame_rate = args.batch_size * float(1 / (end - start))
        avg_frame_rate = (avg_frame_rate * i + running_frame_rate) / (i + 1)
        if i % args.print_every == args.print_every - 1:
            print('epoch: %d, train: %d/%d, '
                  'loss: %.5f, frame: %.2fHz/%.2fHz' % (
                      epoch,
                      i + 1,
                      len(train_loader),
                      running_loss / (i + 1),
                      running_frame_rate,
                      avg_frame_rate
                  ))
    net.train_loss['epoch_loss'].append(running_loss / (i + 1))


def validate(net, valset, val_loader, criterion, epoch, args):
    # switch to 'eval' mode
    net.eval()
    cudnn.benchmark = False

    image_list = valset.image_list

    if args.save_output:
        epoch_result_dir = os.path.join(args.result_dir, str(epoch))
        if not os.path.exists(epoch_result_dir):
            os.makedirs(epoch_result_dir)
        cmap = plt.cm.get_cmap('jet')

    pd_counts = []
    gt_counts = []
    with torch.no_grad():
        avg_frame_rate = 0.0
        for i, sample in enumerate(val_loader):
            torch.cuda.synchronize()
            start = time()

            image, gtcount = sample['image'], sample['gtcount']
            # inference
            output = net(image.cuda(), is_normalize=not args.save_output)
            if args.save_output:
                output_save = output
                # normalization
                output = Normalizer.gpu_normalizer(output, image.size()[2], image.size()[3], args.input_size,
                                                   args.output_stride)
            # postprocessing
            output = np.clip(output, 0, None)

            pdcount = output.sum()
            gtcount = float(gtcount.numpy())

            if args.save_output:
                _, image_name = os.path.split(image_list[i])
                output_save = np.clip(output_save.squeeze().cpu().numpy(), 0, None)
                output_save = recover_countmap(output_save, image, args.input_size, args.output_stride)
                output_save = output_save / (output_save.max() + 1e-12)
                output_save = cmap(output_save) * 255.
                # image composition
                image = valset.images[image_list[i]]
                nh, nw = output_save.shape[:2]
                image = cv2.resize(image, (nw, nh), interpolation=cv2.INTER_CUBIC)
                output_save = 0.5 * image + 0.5 * output_save[:, :, 0:3]

                dotimage = valset.dotimages[image_list[i]]

                fig = plt.figure()
                ax1 = fig.add_subplot(1, 2, 1)
                ax1.imshow(dotimage.astype(np.uint8))
                ax1.get_xaxis().set_visible(False)
                ax1.get_yaxis().set_visible(False)
                ax2 = fig.add_subplot(1, 2, 2)
                ax2.imshow(output_save.astype(np.uint8))
                ax2.get_xaxis().set_visible(False)
                ax2.get_yaxis().set_visible(False)
                fig.suptitle('manual count=%4.2f, inferred count=%4.2f' % (gtcount, pdcount), fontsize=10)
                if args.dataset == 'mtc':
                    plt.tight_layout(rect=[0, 0, 1, 1.4])  # maize tassels counting
                elif args.dataset == 'wec':
                    plt.tight_layout(rect=[0, 0, 1, 1.45])  # wheat ears counting
                elif args.dataset == 'shc':
                    plt.tight_layout(rect=[0, 0, 0.95, 1])  # sorghum heads counting -- dataset1
                    # plt.tight_layout(rect=[0, 0, 1.2, 1]) # sorghum heads counting -- dataset2
                plt.savefig(os.path.join(epoch_result_dir, image_name.replace('.jpg', '.png')), bbox_inches='tight',
                            dpi=300)
                plt.close()

            # compute mae and mse
            pd_counts.append(pdcount)
            gt_counts.append(gtcount)
            mae = compute_mae(pd_counts, gt_counts)
            mse = compute_mse(pd_counts, gt_counts)
            rmae, rmse = compute_relerr(pd_counts, gt_counts)

            torch.cuda.synchronize()
            end = time()

            running_frame_rate = 1 * float(1 / (end - start))
            avg_frame_rate = (avg_frame_rate * i + running_frame_rate) / (i + 1)
            if i % args.print_every == args.print_every - 1:
                print(
                    'epoch: {0}, test: {1}/{2}, pre: {3:.2f}, gt:{4:.2f}, me:{5:.2f}, mae: {6:.2f}, mse: {7:.2f}, rmae: {8:.2f}%, rmse: {9:.2f}%, frame: {10:.2f}Hz/{11:.2f}Hz'
                        .format(epoch, i + 1, len(val_loader), pdcount, gtcount, pdcount - gtcount, mae, mse, rmae,
                                rmse, running_frame_rate, avg_frame_rate)
                )
            start = time()
    r2 = rsquared(pd_counts, gt_counts)
    np.save(args.snapshot_dir + '/pd.npy', pd_counts)
    np.save(args.snapshot_dir + '/gt.npy', gt_counts)
    print('epoch: {0}, mae: {1:.2f}, mse: {2:.2f}, rmae: {3:.2f}%, rmse: {4:.2f}%, r2: {5:.4f}'.format(epoch, mae, mse,
                                                                                                       rmae, rmse, r2))
    # write to files        
    with open(os.path.join(args.snapshot_dir, args.exp + '.txt'), 'a') as f:
        print(
            'epoch: {0}, mae: {1:.2f}, mse: {2:.2f}, rmae: {3:.2f}%, rmse: {4:.2f}%, r2: {5:.4f}'.format(epoch, mae,
                                                                                                         mse, rmae,
                                                                                                         rmse, r2),
            file=f
        )
    with open(os.path.join(args.snapshot_dir, 'counts.txt'), 'a') as f:
        for pd, gt in zip(pd_counts, gt_counts):
            print(
                '{0} {1}'.format(pd, gt),
                file=f
            )
    # save stats
    net.val_loss['epoch_loss'].append(mae)
    net.measure['mae'].append(mae)
    net.measure['mse'].append(mse)
    net.measure['rmae'].append(rmae)
    net.measure['rmse'].append(rmse)
    net.measure['r2'].append(r2)


#### AFTER EDIT

In [30]:
IMG_MEAN = np.array(IMG_MEAN).reshape((1, 1, 3))
IMG_STD = np.array(IMG_STD).reshape((1, 1, 3))

CROP_SIZE = tuple(CROP_SIZE) if len(CROP_SIZE) > 1 else CROP_SIZE

# seeding for reproducbility
#if torch.cuda.is_available():
    #torch.cuda.manual_seed(args.random_seed)
torch.manual_seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)

# instantiate dataset
dataset = dataset_list[DATASET]

SNAPSHOT_DIR = os.path.join(SNAPSHOT_DIR, DATASET.lower(), EXP)
if not os.path.exists(SNAPSHOT_DIR):
    os.makedirs(SNAPSHOT_DIR)

RESULT_DIR = os.path.join(RESULT_DIR, DATASET.lower(), EXP)
if not os.path.exists(RESULT_DIR):
    os.makedirs(RESULT_DIR)

RESTORE_FROM = os.path.join(SNAPSHOT_DIR, RESTORE_FROM)

#arguments = vars(args)
#for item in arguments:
    #print(item, ':\t', arguments[item])

In [ ]:
parser.add_argument("--image-scale", type=float, default=IMG_SCALE, help="Scale factor used in normalization.")
    parser.add_argument("--image-mean", nargs='+', type=float, default=IMG_MEAN, help="Mean used in normalization.")
    parser.add_argument("--image-std", nargs='+', type=float, default=IMG_STD, help="Std used in normalization.")
    parser.add_argument("--scales", type=int, default=SCALES, help="Scales of crop.")
    parser.add_argument("--shorter-side", type=int, default=SHORTER_SIDE, help="Shorter side of the image.")
    # system-related parameters
    parser.add_argument("--data-dir", type=str, default=DATA_DIR, help="Path to the directory containing the dataset.")
    parser.add_argument("--dataset", type=str, default=DATASET, help="Dataset type.")
    parser.add_argument("--exp", type=str, default=EXP, help="Experiment path.")
    parser.add_argument("--data-list", type=str, default=DATA_LIST,
                        help="Path to the file listing the images in the dataset.")
    parser.add_argument("--data-val-list", type=str, default=DATA_VAL_LIST,
                        help="Path to the file listing the images in the val dataset.")
    parser.add_argument("--restore-from", type=str, default=RESTORE_FROM, help="Name of restored model.")
    parser.add_argument("--snapshot-dir", type=str, default=SNAPSHOT_DIR, help="Where to save snapshots of the model.")
    parser.add_argument("--result-dir", type=str, default=RESULT_DIR, help="Where to save inferred results.")
    parser.add_argument("--save-output", action="store_true", help="Whether to save the output.")
    # model-related parameters
    parser.add_argument("--input-size", type=int, default=INPUT_SIZE, help="the minimum input size of the model.")
    parser.add_argument("--output-stride", type=int, default=OUTPUT_STRIDE, help="Output stride of the model.")
    parser.add_argument("--resize-ratio", type=float, default=RESIZE_RATIO, help="Resizing ratio.")
    parser.add_argument("--model", type=str, default=MODEL, help="model to be chosen.")
    parser.add_argument("--use-pretrained", action="store_true", help="Whether to use pretrained model.")
    parser.add_argument("--freeze-bn", action="store_true", help="Whether to freeze encoder bnorm layers.")
    parser.add_argument("--sync-bn", action="store_true", help="Whether to apply synchronized batch normalization.")
    # training-related parameters
    parser.add_argument("--optimizer", type=str, default=OPTIMIZER, choices=['sgd', 'adam'], help="Choose optimizer.")
    parser.add_argument("--batch-size", type=int, default=BATCH_SIZE,
                        help="Number of images sent to the network in one step.")
    parser.add_argument("--milestones", nargs='+', type=int, default=MILESTONES, help="Multistep policy.")
    parser.add_argument("--crop-size", nargs='+', type=int, default=CROP_SIZE, help="Size of crop.")
    parser.add_argument("--evaluate-only", action="store_true", help="Whether to perform evaluation.")
    parser.add_argument("--learning-rate", type=float, default=LEARNING_RATE, help="Base learning rate for training.")
    parser.add_argument("--momentum", type=float, default=MOMENTUM, help="Momentum component of the optimizer.")
    parser.add_argument("--weight-decay", type=float, default=WEIGHT_DECAY,
                        help="Regularisation parameter for L2-loss.")
    parser.add_argument("--mult", type=float, default=MULT, help="LR multiplier for pretrained layers.")
    parser.add_argument("--num-epochs", type=int, default=NUM_EPOCHS, help="Number of training epochs.")
    parser.add_argument("--num-workers", type=int, default=NUM_CPU_WORKERS, help="Number of CPU cores used.")
    parser.add_argument("--print-every", type=int, default=PRINT_EVERY, help="Print information every often.")
    parser.add_argument("--random-seed", type=int, default=RANDOM_SEED,
                        help="Random seed to have reproducible results.")
    parser.add_argument("--val-every", type=int, default=VAL_EVERY, help="How often performing validation.")

In [31]:
args = {
    "image_scale":IMG_SCALE,
    "image_mean":IMG_MEAN,
    "image_std": IMG_STD,
    "scales":SCALES,
    "shorter_side":SHORTER_SIDE,
    "data_dir":DATA_DIR,
    "dataset":DATASET,
    "exp":EXP,
    "data_list":DATA_LIST,
    "data_val_list":DATA_VAL_LIST,
    "restore_from":RESTORE_FROM,
    "snapshot_dir":SNAPSHOT_DIR,
    "result_dir":RESULT_DIR,
    "save_output":"store_true",
    "input_size":INPUT_SIZE,
    "output_stride":OUTPUT_STRIDE,
    "resize_ratio":RESIZE_RATIO,
    "model":MODEL,
    "optimizer":OPTIMIZER,
    "batch_size":BATCH_SIZE,
    "milestones":MILESTONES,
    "crop_size":CROP_SIZE,
    "evaluate_only":"store_true",
    "learning_rate":LEARNING_RATE,
    "momentum":MOMENTUM,
    "weight_decay":WEIGHT_DECAY,
    "mult":MULT,
    "num_epochs":NUM_EPOCHS,
    "num_workers":NUM_CPU_WORKERS,
    "print_every":PRINT_EVERY,
    "random_seed":RANDOM_SEED,
    "val_every":VAL_EVERY
}

print(args)

{'image_scale': 0.00392156862745098, 'image_mean': array([[[0.3405, 0.4747, 0.2418]]]), 'image_std': array([[[1, 1, 1]]]), 'scales': [0.7, 1, 1.3], 'shorter_side': 224, 'data_dir': 'maize_counting_dataset', 'dataset': 'mtc', 'exp': 'tasselnetv2_rf110_i64o8_r0125_crop256_lr-2_bs9_epoch500', 'data_list': 'maize_counting_dataset/train.txt', 'data_val_list': 'maize_counting_dataset/test.txt', 'restore_from': './snapshots\\mtc\\tasselnetv2_rf110_i64o8_r0125_crop256_lr-2_bs9_epoch500\\model_best.pth.tar', 'snapshot_dir': './snapshots\\mtc\\tasselnetv2_rf110_i64o8_r0125_crop256_lr-2_bs9_epoch500', 'result_dir': './results\\mtc\\tasselnetv2_rf110_i64o8_r0125_crop256_lr-2_bs9_epoch500', 'save_output': 'store_true', 'input_size': 64, 'output_stride': 8, 'resize_ratio': 0.125, 'model': 'tasselnetv2', 'optimizer': 'sgd', 'batch_size': 9, 'milestones': [200, 400], 'crop_size': (256, 256), 'evaluate_only': 'store_true', 'learning_rate': 0.01, 'momentum': 0.95, 'weight_decay': 0.0005, 'mult': 1, 'num

In [45]:
#args = get_arguments()

# args.evaluate_only = True
# args.save_output = True

args.image_mean = np.array(args.image_mean).reshape((1, 1, 3))
args.image_std = np.array(args.image_std).reshape((1, 1, 3))

args.crop_size = tuple(args.crop_size) if len(args.crop_size) > 1 else args.crop_size

# seeding for reproducbility
if torch.cuda.is_available():
    torch.cuda.manual_seed(args.random_seed)
torch.manual_seed(args.random_seed)
np.random.seed(args.random_seed)

# instantiate dataset
dataset = dataset_list[args.dataset]

args.snapshot_dir = os.path.join(args.snapshot_dir, args.dataset.lower(), args.exp)
if not os.path.exists(args.snapshot_dir):
    os.makedirs(args.snapshot_dir)

args.result_dir = os.path.join(args.result_dir, args.dataset.lower(), args.exp)
if not os.path.exists(args.result_dir):
    os.makedirs(args.result_dir)

args.restore_from = os.path.join(args.snapshot_dir, args.restore_from)

arguments = vars(args)
for item in arguments:
    print(item, ':\t', arguments[item])

AttributeError: 'dict' object has no attribute 'image_mean'

#### AFTER EDIT

In [32]:
# instantiate network
net = CountingModels(
      arc=MODEL,
      input_size=INPUT_SIZE,
      output_stride=OUTPUT_STRIDE
    )

net = nn.DataParallel(net)
#net.cuda()

# filter parameters
learning_params = [p[1] for p in net.named_parameters()]
pretrained_params = []

# define loss function and optimizer
#criterion = nn.L1Loss(reduction='mean').cuda()
criterion = nn.L1Loss(reduction='mean')

if OPTIMIZER == 'sgd':
    optimizer = torch.optim.SGD(
        [
            {'params': learning_params},
            {'params': pretrained_params, 'lr': LEARNING_RATE / MULT},
        ],
        lr=LEARNING_RATE,
        momentum=MOMENTUM,
        weight_decay=WEIGHT_DECAY
    )
elif args.optimizer == 'adam':
    optimizer = torch.optim.Adam(
        [
            {'params': learning_params},
            {'params': pretrained_params, 'lr': LEARNING_RATE / args.mult},
        ],
        lr=LEARNING_RATE
    )
else:
    raise NotImplementedError


In [33]:
# restore parameters
start_epoch = 0
net.train_loss = {
    'running_loss': [],
    'epoch_loss': []
}
net.val_loss = {
    'running_loss': [],
    'epoch_loss': []
}
net.measure = {
    'mae': [],
    'mse': [],
    'rmae': [],
    'rmse': [],
    'r2': []
}
if RESTORE_FROM is not None:
    if os.path.isfile(RESTORE_FROM):
        checkpoint = torch.load(RESTORE_FROM)
        net.load_state_dict(checkpoint['state_dict'])
        if 'epoch' in checkpoint:
            start_epoch = checkpoint['epoch']
        if 'optimizer' in checkpoint:
            optimizer.load_state_dict(checkpoint['optimizer'])
        if 'train_loss' in checkpoint:
            net.train_loss = checkpoint['train_loss']
        if 'val_loss' in checkpoint:
            net.val_loss = checkpoint['val_loss']
        if 'measure' in checkpoint:
            net.measure['mae'] = checkpoint['measure']['mae'] if 'mae' in checkpoint['measure'] else []
            net.measure['mse'] = checkpoint['measure']['mse'] if 'mse' in checkpoint['measure'] else []
            net.measure['rmae'] = checkpoint['measure']['rmae'] if 'rmae' in checkpoint['measure'] else []
            net.measure['rmse'] = checkpoint['measure']['rmse'] if 'rmse' in checkpoint['measure'] else []
            net.measure['r2'] = checkpoint['measure']['r2'] if 'r2' in checkpoint['measure'] else []
        print("==> load checkpoint '{}' (epoch {})"
              .format(RESTORE_FROM, start_epoch))
    else:
        with open(os.path.join(SNAPSHOT_DIR, EXP + '.txt'), 'a') as f:
            for item in args:
                print(item, ':\t', args[item], file=f)
        print("==> no checkpoint found at '{}'".format(RESTORE_FROM))


==> no checkpoint found at './snapshots\mtc\tasselnetv2_rf110_i64o8_r0125_crop256_lr-2_bs9_epoch500\model_best.pth.tar'


In [34]:
# define transform
transform_train = [
    RandomCrop(CROP_SIZE),
    RandomFlip(),
    Normalize(
        IMG_SCALE,
        IMG_MEAN,
        IMG_STD
    ),
    ToTensor(),
    ZeroPadding(OUTPUT_STRIDE)
]
transform_val = [
    Normalize(
        IMG_SCALE,
        IMG_MEAN,
        IMG_STD
    ),
    ToTensor(),
    ZeroPadding(OUTPUT_STRIDE)
]
composed_transform_train = transforms.Compose(transform_train)
composed_transform_val = transforms.Compose(transform_val)

# define dataset loader
trainset = dataset(
    data_dir=DATA_DIR,
    data_list=DATA_LIST,
    ratio=RESIZE_RATIO,
    train=True,
    transform=composed_transform_train
)
train_loader = DataLoader(
    trainset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=NUM_CPU_WORKERS,
    pin_memory=True,
    drop_last=True
)
valset = dataset(
    data_dir=DATA_DIR,
    data_list=DATA_LIST,
    ratio=RESIZE_RATIO,
    train=False,
    transform=composed_transform_val
)
val_loader = DataLoader(
    valset,
    batch_size=1,
    shuffle=False,
    num_workers=NUM_CPU_WORKERS,
    pin_memory=True
)

In [39]:
validate(net, valset, val_loader, criterion, start_epoch, args)

<ipython-input-17-9a25d3e4bf17>:60: RuntimeWarning: divide by zero encountered in true_divide
  idx_img = 1/idx_img


epoch: 0, test: 1/186, pre: inf, gt:97.00, me:inf, mae: inf, mse: inf, rmae: inf%, rmse: inf%, frame: 0.49Hz/0.49Hz
epoch: 0, test: 2/186, pre: inf, gt:40.00, me:inf, mae: inf, mse: inf, rmae: inf%, rmse: inf%, frame: 0.76Hz/0.63Hz
epoch: 0, test: 3/186, pre: inf, gt:23.00, me:inf, mae: inf, mse: inf, rmae: inf%, rmse: inf%, frame: 0.77Hz/0.67Hz
epoch: 0, test: 4/186, pre: inf, gt:44.00, me:inf, mae: inf, mse: inf, rmae: inf%, rmse: inf%, frame: 0.80Hz/0.71Hz
epoch: 0, test: 5/186, pre: inf, gt:32.00, me:inf, mae: inf, mse: inf, rmae: inf%, rmse: inf%, frame: 0.81Hz/0.73Hz
epoch: 0, test: 6/186, pre: inf, gt:49.00, me:inf, mae: inf, mse: inf, rmae: inf%, rmse: inf%, frame: 0.88Hz/0.75Hz
epoch: 0, test: 7/186, pre: inf, gt:41.00, me:inf, mae: inf, mse: inf, rmae: inf%, rmse: inf%, frame: 0.80Hz/0.76Hz
epoch: 0, test: 8/186, pre: inf, gt:60.00, me:inf, mae: inf, mse: inf, rmae: inf%, rmse: inf%, frame: 0.96Hz/0.78Hz
epoch: 0, test: 9/186, pre: inf, gt:32.00, me:inf, mae: inf, mse: inf, r

epoch: 0, test: 72/186, pre: inf, gt:18.00, me:inf, mae: inf, mse: inf, rmae: inf%, rmse: inf%, frame: 0.76Hz/0.79Hz
epoch: 0, test: 73/186, pre: inf, gt:64.00, me:inf, mae: inf, mse: inf, rmae: inf%, rmse: inf%, frame: 0.79Hz/0.79Hz
epoch: 0, test: 74/186, pre: inf, gt:36.00, me:inf, mae: inf, mse: inf, rmae: inf%, rmse: inf%, frame: 0.76Hz/0.79Hz
epoch: 0, test: 75/186, pre: inf, gt:35.00, me:inf, mae: inf, mse: inf, rmae: inf%, rmse: inf%, frame: 0.82Hz/0.79Hz
epoch: 0, test: 76/186, pre: inf, gt:18.00, me:inf, mae: inf, mse: inf, rmae: inf%, rmse: inf%, frame: 0.95Hz/0.79Hz
epoch: 0, test: 77/186, pre: inf, gt:14.00, me:inf, mae: inf, mse: inf, rmae: inf%, rmse: inf%, frame: 0.84Hz/0.79Hz
epoch: 0, test: 78/186, pre: inf, gt:13.00, me:inf, mae: inf, mse: inf, rmae: inf%, rmse: inf%, frame: 0.78Hz/0.79Hz
epoch: 0, test: 79/186, pre: inf, gt:96.00, me:inf, mae: inf, mse: inf, rmae: inf%, rmse: inf%, frame: 0.78Hz/0.79Hz
epoch: 0, test: 80/186, pre: inf, gt:9.00, me:inf, mae: inf, mse

epoch: 0, test: 142/186, pre: inf, gt:96.00, me:inf, mae: inf, mse: inf, rmae: inf%, rmse: inf%, frame: 0.80Hz/0.79Hz
epoch: 0, test: 143/186, pre: inf, gt:18.00, me:inf, mae: inf, mse: inf, rmae: inf%, rmse: inf%, frame: 0.78Hz/0.79Hz
epoch: 0, test: 144/186, pre: inf, gt:100.00, me:inf, mae: inf, mse: inf, rmae: inf%, rmse: inf%, frame: 0.80Hz/0.79Hz
epoch: 0, test: 145/186, pre: inf, gt:64.00, me:inf, mae: inf, mse: inf, rmae: inf%, rmse: inf%, frame: 0.78Hz/0.79Hz
epoch: 0, test: 146/186, pre: inf, gt:48.00, me:inf, mae: inf, mse: inf, rmae: inf%, rmse: inf%, frame: 0.78Hz/0.79Hz
epoch: 0, test: 147/186, pre: inf, gt:5.00, me:inf, mae: inf, mse: inf, rmae: inf%, rmse: inf%, frame: 0.78Hz/0.79Hz
epoch: 0, test: 148/186, pre: inf, gt:65.00, me:inf, mae: inf, mse: inf, rmae: inf%, rmse: inf%, frame: 0.77Hz/0.79Hz
epoch: 0, test: 149/186, pre: inf, gt:53.00, me:inf, mae: inf, mse: inf, rmae: inf%, rmse: inf%, frame: 0.82Hz/0.79Hz
epoch: 0, test: 150/186, pre: inf, gt:52.00, me:inf, mae

D:\anaconda3\lib\site-packages\numpy\lib\function_base.py:2474: RuntimeWarning: invalid value encountered in subtract
  X -= avg[:, None]


NameError: name 'SNAPSHOT_DIRr' is not defined

In [47]:
print('alchemy start...')
#if evaluate_only:
#validate(net, valset, val_loader, criterion, start_epoch, args)
    #return

best_mae = 1000000.0
resume_epoch = -1 if start_epoch == 0 else start_epoch
scheduler = MultiStepLR(optimizer, milestones=MILESTONES, gamma=0.1, last_epoch=resume_epoch)
for epoch in range(start_epoch, NUM_EPOCHS):
    # train
    train(net, train_loader, criterion, optimizer, epoch + 1, args)
    if epoch % VAL_EVERY == VAL_EVERY - 1:
        # val
        validate(net, valset, val_loader, criterion, epoch + 1, args)
        # save_checkpoint
        state = {
            'state_dict': net.state_dict(),
            'optimizer': optimizer.state_dict(),
            'epoch': epoch + 1,
            'train_loss': net.train_loss,
            'val_loss': net.val_loss,
            'measure': net.measure
        }
        save_checkpoint(state, SNAPSHOT_DIR, filename='model_ckpt.pth.tar')
        if net.measure['mae'][-1] <= best_mae:
            save_checkpoint(state, SNAPSHOT_DIR, filename='model_best.pth.tar')
            best_mae = net.measure['mae'][-1]
            best_mse = net.measure['mse'][-1]
            best_rmae = net.measure['rmae'][-1]
            best_rmse = net.measure['rmse'][-1]
            best_r2 = net.measure['r2'][-1]
        print(EXP + ' epoch {} finished!'.format(epoch + 1))
        print('best mae: {0:.2f}, best mse: {1:.2f}, best_rmae: {2:.2f}, best_rmse: {3:.2f}, best_r2: {4:.4f}'
              .format(best_mae, best_mse, best_rmae, best_rmse, best_r2))
        plot_learning_curves(net, SNAPSHOT_DIR)
    scheduler.step()

print('Experiments with ' + EXP + ' done!')
with open(os.path.join(SNAPSHOT_DIR, EXP + '.txt'), 'a') as f:
    print(
        'best mae: {0:.2f}, best mse: {1:.2f}, best_rmae: {2:.2f}, best_rmse: {3:.2f}, best_r2: {4:.4f}'
            .format(best_mae, best_mse, best_rmae, best_rmse, best_r2),
        file=f
    )
    print(
        'overall best mae: {0:.2f}, overall best mse: {1:.2f}, overall best_rmae: {2:.2f}, overall best_rmse: {3:.2f}, overall best_r2: {4:.4f}'
            .format(min(net.measure['mae']), min(net.measure['mse']), min(net.measure['rmae']),
                    min(net.measure['rmse']), max(net.measure['r2'])),
        file=f
    )

alchemy start...
epoch: 1, train: 1/20, loss: 0.44639, frame: 16.20Hz/16.20Hz
epoch: 1, train: 2/20, loss: 0.56881, frame: 15.53Hz/15.86Hz
epoch: 1, train: 3/20, loss: 0.62039, frame: 12.29Hz/14.67Hz
epoch: 1, train: 4/20, loss: 0.61372, frame: 14.25Hz/14.57Hz
epoch: 1, train: 5/20, loss: 0.57272, frame: 14.15Hz/14.48Hz
epoch: 1, train: 6/20, loss: 0.55888, frame: 13.88Hz/14.38Hz
epoch: 1, train: 7/20, loss: 0.54104, frame: 14.86Hz/14.45Hz
epoch: 1, train: 8/20, loss: 0.52805, frame: 14.66Hz/14.48Hz
epoch: 1, train: 9/20, loss: 0.51376, frame: 13.45Hz/14.36Hz
epoch: 1, train: 10/20, loss: 0.50144, frame: 15.13Hz/14.44Hz
epoch: 1, train: 11/20, loss: 0.49109, frame: 12.06Hz/14.22Hz
epoch: 1, train: 12/20, loss: 0.49254, frame: 11.63Hz/14.01Hz
epoch: 1, train: 13/20, loss: 0.48154, frame: 14.33Hz/14.03Hz
epoch: 1, train: 14/20, loss: 0.47506, frame: 13.45Hz/13.99Hz
epoch: 1, train: 15/20, loss: 0.46798, frame: 11.21Hz/13.80Hz
epoch: 1, train: 16/20, loss: 0.46991, frame: 10.99Hz/13.63Hz


<ipython-input-17-9a25d3e4bf17>:60: RuntimeWarning: divide by zero encountered in true_divide
  idx_img = 1/idx_img


epoch: 1, test: 1/186, pre: inf, gt:97.00, me:inf, mae: inf, mse: inf, rmae: inf%, rmse: inf%, frame: 0.71Hz/0.71Hz
epoch: 1, test: 2/186, pre: inf, gt:40.00, me:inf, mae: inf, mse: inf, rmae: inf%, rmse: inf%, frame: 0.68Hz/0.69Hz
epoch: 1, test: 3/186, pre: inf, gt:23.00, me:inf, mae: inf, mse: inf, rmae: inf%, rmse: inf%, frame: 0.56Hz/0.65Hz
epoch: 1, test: 4/186, pre: inf, gt:44.00, me:inf, mae: inf, mse: inf, rmae: inf%, rmse: inf%, frame: 0.71Hz/0.66Hz
epoch: 1, test: 5/186, pre: inf, gt:32.00, me:inf, mae: inf, mse: inf, rmae: inf%, rmse: inf%, frame: 0.69Hz/0.67Hz
epoch: 1, test: 6/186, pre: inf, gt:49.00, me:inf, mae: inf, mse: inf, rmae: inf%, rmse: inf%, frame: 0.83Hz/0.70Hz
epoch: 1, test: 7/186, pre: inf, gt:41.00, me:inf, mae: inf, mse: inf, rmae: inf%, rmse: inf%, frame: 0.71Hz/0.70Hz
epoch: 1, test: 8/186, pre: inf, gt:60.00, me:inf, mae: inf, mse: inf, rmae: inf%, rmse: inf%, frame: 0.86Hz/0.72Hz
epoch: 1, test: 9/186, pre: inf, gt:32.00, me:inf, mae: inf, mse: inf, r

epoch: 1, test: 72/186, pre: inf, gt:18.00, me:inf, mae: inf, mse: inf, rmae: inf%, rmse: inf%, frame: 0.71Hz/0.71Hz
epoch: 1, test: 73/186, pre: inf, gt:64.00, me:inf, mae: inf, mse: inf, rmae: inf%, rmse: inf%, frame: 0.67Hz/0.71Hz
epoch: 1, test: 74/186, pre: inf, gt:36.00, me:inf, mae: inf, mse: inf, rmae: inf%, rmse: inf%, frame: 0.70Hz/0.71Hz
epoch: 1, test: 75/186, pre: inf, gt:35.00, me:inf, mae: inf, mse: inf, rmae: inf%, rmse: inf%, frame: 0.74Hz/0.71Hz
epoch: 1, test: 76/186, pre: inf, gt:18.00, me:inf, mae: inf, mse: inf, rmae: inf%, rmse: inf%, frame: 0.88Hz/0.71Hz
epoch: 1, test: 77/186, pre: inf, gt:14.00, me:inf, mae: inf, mse: inf, rmae: inf%, rmse: inf%, frame: 0.75Hz/0.71Hz
epoch: 1, test: 78/186, pre: inf, gt:13.00, me:inf, mae: inf, mse: inf, rmae: inf%, rmse: inf%, frame: 0.72Hz/0.71Hz
epoch: 1, test: 79/186, pre: inf, gt:96.00, me:inf, mae: inf, mse: inf, rmae: inf%, rmse: inf%, frame: 0.72Hz/0.71Hz
epoch: 1, test: 80/186, pre: inf, gt:9.00, me:inf, mae: inf, mse

epoch: 1, test: 142/186, pre: inf, gt:96.00, me:inf, mae: inf, mse: inf, rmae: inf%, rmse: inf%, frame: 0.77Hz/0.71Hz
epoch: 1, test: 143/186, pre: inf, gt:18.00, me:inf, mae: inf, mse: inf, rmae: inf%, rmse: inf%, frame: 0.66Hz/0.71Hz
epoch: 1, test: 144/186, pre: inf, gt:100.00, me:inf, mae: inf, mse: inf, rmae: inf%, rmse: inf%, frame: 0.73Hz/0.71Hz
epoch: 1, test: 145/186, pre: inf, gt:64.00, me:inf, mae: inf, mse: inf, rmae: inf%, rmse: inf%, frame: 0.69Hz/0.71Hz
epoch: 1, test: 146/186, pre: inf, gt:48.00, me:inf, mae: inf, mse: inf, rmae: inf%, rmse: inf%, frame: 0.71Hz/0.71Hz
epoch: 1, test: 147/186, pre: inf, gt:5.00, me:inf, mae: inf, mse: inf, rmae: inf%, rmse: inf%, frame: 0.72Hz/0.71Hz
epoch: 1, test: 148/186, pre: inf, gt:65.00, me:inf, mae: inf, mse: inf, rmae: inf%, rmse: inf%, frame: 0.72Hz/0.71Hz
epoch: 1, test: 149/186, pre: inf, gt:53.00, me:inf, mae: inf, mse: inf, rmae: inf%, rmse: inf%, frame: 0.76Hz/0.71Hz
epoch: 1, test: 150/186, pre: inf, gt:52.00, me:inf, mae

NameError: name 'best_mse' is not defined

In [ ]:
# instantiate network
net = CountingModels(
      arc=args.model,
      input_size=args.input_size,
      output_stride=args.output_stride
    )

net = nn.DataParallel(net)
#net.cuda()

# filter parameters
learning_params = [p[1] for p in net.named_parameters()]
pretrained_params = []

# define loss function and optimizer
criterion = nn.L1Loss(reduction='mean').cuda()

if args.optimizer == 'sgd':
    optimizer = torch.optim.SGD(
        [
            {'params': learning_params},
            {'params': pretrained_params, 'lr': args.learning_rate / args.mult},
        ],
        lr=args.learning_rate,
        momentum=args.momentum,
        weight_decay=args.weight_decay
    )
elif args.optimizer == 'adam':
    optimizer = torch.optim.Adam(
        [
            {'params': learning_params},
            {'params': pretrained_params, 'lr': args.learning_rate / args.mult},
        ],
        lr=args.learning_rate
    )
else:
    raise NotImplementedError

# restore parameters
start_epoch = 0
net.train_loss = {
    'running_loss': [],
    'epoch_loss': []
}
net.val_loss = {
    'running_loss': [],
    'epoch_loss': []
}
net.measure = {
    'mae': [],
    'mse': [],
    'rmae': [],
    'rmse': [],
    'r2': []
}
if args.restore_from is not None:
    if os.path.isfile(args.restore_from):
        checkpoint = torch.load(args.restore_from)
        net.load_state_dict(checkpoint['state_dict'])
        if 'epoch' in checkpoint:
            start_epoch = checkpoint['epoch']
        if 'optimizer' in checkpoint:
            optimizer.load_state_dict(checkpoint['optimizer'])
        if 'train_loss' in checkpoint:
            net.train_loss = checkpoint['train_loss']
        if 'val_loss' in checkpoint:
            net.val_loss = checkpoint['val_loss']
        if 'measure' in checkpoint:
            net.measure['mae'] = checkpoint['measure']['mae'] if 'mae' in checkpoint['measure'] else []
            net.measure['mse'] = checkpoint['measure']['mse'] if 'mse' in checkpoint['measure'] else []
            net.measure['rmae'] = checkpoint['measure']['rmae'] if 'rmae' in checkpoint['measure'] else []
            net.measure['rmse'] = checkpoint['measure']['rmse'] if 'rmse' in checkpoint['measure'] else []
            net.measure['r2'] = checkpoint['measure']['r2'] if 'r2' in checkpoint['measure'] else []
        print("==> load checkpoint '{}' (epoch {})"
              .format(args.restore_from, start_epoch))
    else:
        with open(os.path.join(args.snapshot_dir, args.exp + '.txt'), 'a') as f:
            for item in arguments:
                print(item, ':\t', arguments[item], file=f)
        print("==> no checkpoint found at '{}'".format(args.restore_from))

# define transform
transform_train = [
    RandomCrop(args.crop_size),
    RandomFlip(),
    Normalize(
        args.image_scale,
        args.image_mean,
        args.image_std
    ),
    ToTensor(),
    ZeroPadding(args.output_stride)
]
transform_val = [
    Normalize(
        args.image_scale,
        args.image_mean,
        args.image_std
    ),
    ToTensor(),
    ZeroPadding(args.output_stride)
]
composed_transform_train = transforms.Compose(transform_train)
composed_transform_val = transforms.Compose(transform_val)

# define dataset loader
trainset = dataset(
    data_dir=args.data_dir,
    data_list=args.data_list,
    ratio=args.resize_ratio,
    train=True,
    transform=composed_transform_train
)
train_loader = DataLoader(
    trainset,
    batch_size=args.batch_size,
    shuffle=True,
    num_workers=args.num_workers,
    pin_memory=True,
    drop_last=True
)
valset = dataset(
    data_dir=args.data_dir,
    data_list=args.data_val_list,
    ratio=args.resize_ratio,
    train=False,
    transform=composed_transform_val
)
val_loader = DataLoader(
    valset,
    batch_size=1,
    shuffle=False,
    num_workers=args.num_workers,
    pin_memory=True
)

print('alchemy start...')
if args.evaluate_only:
    validate(net, valset, val_loader, criterion, start_epoch, args)
    return

best_mae = 1000000.0
resume_epoch = -1 if start_epoch == 0 else start_epoch
scheduler = MultiStepLR(optimizer, milestones=args.milestones, gamma=0.1, last_epoch=resume_epoch)
for epoch in range(start_epoch, args.num_epochs):
    # train
    train(net, train_loader, criterion, optimizer, epoch + 1, args)
    if epoch % args.val_every == args.val_every - 1:
        # val
        validate(net, valset, val_loader, criterion, epoch + 1, args)
        # save_checkpoint
        state = {
            'state_dict': net.state_dict(),
            'optimizer': optimizer.state_dict(),
            'epoch': epoch + 1,
            'train_loss': net.train_loss,
            'val_loss': net.val_loss,
            'measure': net.measure
        }
        save_checkpoint(state, args.snapshot_dir, filename='model_ckpt.pth.tar')
        if net.measure['mae'][-1] <= best_mae:
            save_checkpoint(state, args.snapshot_dir, filename='model_best.pth.tar')
            best_mae = net.measure['mae'][-1]
            best_mse = net.measure['mse'][-1]
            best_rmae = net.measure['rmae'][-1]
            best_rmse = net.measure['rmse'][-1]
            best_r2 = net.measure['r2'][-1]
        print(args.exp + ' epoch {} finished!'.format(epoch + 1))
        print('best mae: {0:.2f}, best mse: {1:.2f}, best_rmae: {2:.2f}, best_rmse: {3:.2f}, best_r2: {4:.4f}'
              .format(best_mae, best_mse, best_rmae, best_rmse, best_r2))
        plot_learning_curves(net, args.snapshot_dir)
    scheduler.step()

print('Experiments with ' + args.exp + ' done!')
with open(os.path.join(args.snapshot_dir, args.exp + '.txt'), 'a') as f:
    print(
        'best mae: {0:.2f}, best mse: {1:.2f}, best_rmae: {2:.2f}, best_rmse: {3:.2f}, best_r2: {4:.4f}'
            .format(best_mae, best_mse, best_rmae, best_rmse, best_r2),
        file=f
    )
    print(
        'overall best mae: {0:.2f}, overall best mse: {1:.2f}, overall best_rmae: {2:.2f}, overall best_rmse: {3:.2f}, overall best_r2: {4:.4f}'
            .format(min(net.measure['mae']), min(net.measure['mse']), min(net.measure['rmae']),
                    min(net.measure['rmse']), max(net.measure['r2'])),
        file=f
    )